# Use Bootstrap Aggregating
* Train each classifier on a subset the WSIs.Not true bagging because no replacement. Can't be true, because there isn't a fixed dateset.
* Try training each classifier on half, quarter of the dataset.

In [1]:
import os
import sys
import glob
import random
import pickle
import numpy as np
from PIL import Image
import time
import copy
from io import BytesIO
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data
import torchvision.models as models
from torchvision import datasets, models, transforms
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

SEED = 101
np.random.seed(SEED)
from torchvision import datasets, models, transforms

%reload_ext autoreload
%autoreload 2
%matplotlib inline

# Add the src directory for functions
src_dir = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), 'src')
print(src_dir)
sys.path.append(src_dir)

# import my functions:
from WSI_pytorch_utils import*

fast_ai_dir = '/media/rene/Data/fastai/'
sys.path.append(fast_ai_dir)

torch.cuda.set_device(1)
print(torch.cuda.is_available())
print(torch.cuda.current_device())

/media/rene/Data/camelyon/src
True
1


## Train the classifiers

### Try training with normal batch generator

In [7]:
save_path = '/media/rene/Data/camelyon_out/inline_batch_random/resnet50_10e_full_'
batch_size = 64
data_loc = '/media/rene/Data/camelyon_out/tiles_224_100t'
epochs = 10


data_transforms = {
    'train': transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

image_datasets = {x: datasets.ImageFolder(os.path.join(data_loc, x),
                                          data_transforms[x])
                  for x in ['train', 'valid']}

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size,
                                             shuffle=True, num_workers=6)
              for x in ['train', 'valid']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'valid']}
class_names = image_datasets['train'].classes

In [ ]:
print(dataset_sizes)
model_ft = models.resnet50(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 2)
model_ft = model_ft.cuda()

criterion = nn.CrossEntropyLoss()

# all parameters are being optimized
#     optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
optimizer_ft = optim.Adam(model_ft.parameters())


exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=3, gamma=0.1)

model_ft = train_model(model_ft, dataloaders, dataset_sizes, criterion, optimizer_ft, exp_lr_scheduler,
                   num_epochs=epochs)

torch.save(model_ft.state_dict(), save_path+'normal_batch_gen.h5')

#### Get acc

In [4]:
def test_accu(model, test_loader, ttv='train'):
    model.eval()
    correct_count = 0.0
    for i, data in enumerate(test_loader, 0):
        img, labels = data
        x = Variable(img, volatile=True)
        y = Variable(labels, volatile=True)
        if torch.cuda.is_available():
            x = x.cuda()
            y = y.cuda()
            x=torch.squeeze(x)
        outs = model(x)
        _, pred = torch.max(outs, -1)
        correct_count += (pred == y).sum().data[0]
        del x, y, pred, outs
    print('Accu: {}'.format(correct_count/dataset_sizes[ttv]))

In [14]:
model_ft= models.resnet50(pretrained=False)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 2)
model_ft = model_ft.cuda()

model_ft.load_state_dict(torch.load('/media/rene/Data/camelyon_out/inline_batch_random/resnet50_10e_full_normal_batch_gen.h5'))
model_ft.eval()
model_ft.train(False)

test_accu(model_ft, test_loader = dataloaders['train'], ttv='train')

Accu: 0.9285234093637454


In [16]:
model_ft = models.resnet50(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 2)
model_ft = model_ft.cuda()

model_ft.load_state_dict(torch.load('/media/rene/Data/camelyon_out/inline_batch_random/resnet50_10e_full_normal_batch_gen.h5'))
model_ft.eval()
model_ft.train(False)

test_accu(model_ft, test_loader = dataloaders['valid'], ttv='valid')

Accu: 0.922814371257485


## Online batches 
* Create dataloaders

In [2]:
data_loc = '/media/rene/Data/CAMELYON16/TrainingData'
batch_size = 80
dataset_sizes = {}
dataset_sizes['train'] = 100000
dataset_sizes['valid'] = 10000

ttv_split = np.load('/media/rene/Data/camelyon/other/ttv_split.p')
normal_valid = ttv_split['normal_vaild_idx']
tumor_valid = ttv_split['tumor_vaild_idx']
normal_train = list(range(1, 161))
normal_train = [num for num in normal_train if num not in normal_valid]
tumor_train = list(range(1, 111))
tumor_train = [num for num in tumor_train if num not in tumor_valid]

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

train_dataset = WSIDataset(data_loc, normal_train, tumor_train, batch_size, length=dataset_sizes['train'], transforms=data_transforms['train'])
valid_dataset = WSIDataset(data_loc, normal_valid, tumor_valid, batch_size, length=dataset_sizes['valid'], transforms=data_transforms['valid'])

dataloaders ={}
# batch size of 1 because of the weird get item
dataloaders['train'] = DataLoader(train_dataset,  batch_size=1, num_workers=6, shuffle=False)
dataloaders['valid'] = DataLoader(valid_dataset, batch_size=1, num_workers=6, shuffle=False)

model_ft = models.resnet50(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 2)
model_ft = model_ft.cuda()

criterion = nn.CrossEntropyLoss()
epochs = 10

In [ ]:
save_path = '/media/rene/Data/camelyon_out/inline_batch_random/resnet50_10e_full_'

model_ft = models.resnet50(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 2)
model_ft = model_ft.cuda()

criterion = nn.CrossEntropyLoss()

# all parameters are being optimized
#     optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
optimizer_ft = optim.Adam(model_ft.parameters())

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=3, gamma=0.1)

model_ft = train_model(model_ft, dataloaders, dataset_sizes, criterion, optimizer_ft, exp_lr_scheduler,
                   num_epochs=epochs)

torch.save(model_ft.state_dict(), save_path+'online_batch_gen.h5')

#### Performance

In [5]:
model_ft= models.resnet50(pretrained=False)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 2)
model_ft = model_ft.cuda()

model_ft.load_state_dict(torch.load('/media/rene/Data/camelyon_out/inline_batch_random/resnet50_10e_full_online_batch_gen.h5'))
model_ft.eval()
model_ft.train(False)

test_accu(model_ft, test_loader = dataloaders['train'], ttv='train')
test_accu(model_ft, test_loader = dataloaders['valid'], ttv='valid')

Accu: 0.74497
Accu: 0.7217


#### Try standard dataloader

In [6]:
batch_size = 64
data_loc = '/media/rene/Data/camelyon_out/tiles_224_100t'

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

image_datasets = {x: datasets.ImageFolder(os.path.join(data_loc, x),
                                          data_transforms[x])
                  for x in ['train', 'valid']}

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size,
                                             shuffle=True, num_workers=6)
              for x in ['train', 'valid']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'valid']}
class_names = image_datasets['train'].classes

model_ft= models.resnet50(pretrained=False)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 2)
model_ft = model_ft.cuda()

model_ft.load_state_dict(torch.load('/media/rene/Data/camelyon_out/inline_batch_random/resnet50_10e_full_online_batch_gen.h5'))
model_ft.eval()
model_ft.train(False)

test_accu(model_ft, test_loader = dataloaders['train'], ttv='train')
test_accu(model_ft, test_loader = dataloaders['valid'], ttv='valid')

Accu: 0.7147298919567827
Accu: 0.7641916167664671


## Line by line check of sampling locations? 
* Maybe the different masks for the normal in the two versions caused a problem

## Bagging

In [ ]:
save_path = '/media/rene/Data/camelyon_out/online_batch/resnet50_10e_full_'

for i in range(5):
    train_dataset = WSIDataset(data_loc, normal_train, tumor_train, batch_size, length=dataset_sizes['train'], transforms=data_transforms['train'])
    valid_dataset = WSIDataset(data_loc, normal_valid, tumor_valid, batch_size, length=dataset_sizes['valid'], transforms=data_transforms['valid'])

    dataloaders ={}
    # batch size of 1 because of the weird get item
    dataloaders['train'] = DataLoader(train_dataset,  batch_size=1, num_workers=6, shuffle=False)
    dataloaders['valid'] = DataLoader(valid_dataset, batch_size=1, num_workers=6, shuffle=False)

    model_ft = models.resnet50(pretrained=True)
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs, 2)
    model_ft = model_ft.cuda()

    criterion = nn.CrossEntropyLoss()

    # all parameters are being optimized
    optimizer_ft = optim.Adam(model_ft.parameters())
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=3, gamma=0.1)

    model_ft = train_model(model_ft, dataloaders, dataset_sizes, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=epochs)
    
    torch.save(model_ft.state_dict(), save_path+str(i)+'.h5')

  0%|          | 0/1250 [00:00<?, ?it/s]

Epoch 0/9
----------


  2%|▏         | 20/1250 [00:22<23:01,  1.12s/it] 

In [ ]:
save_path = '/media/rene/Data/camelyon_out/inline_batch_random/resnet50_10e_half_'

for i in range(5):
    normal_train_subset = random.sample(normal_train, int(len(normal_train)/2))

    train_dataset = WSIDataset(data_loc, normal_train_subset, tumor_train, batch_size, length=dataset_sizes['train'], transforms=data_transforms['train'])
    valid_dataset = WSIDataset(data_loc, normal_valid, tumor_valid, batch_size, length=dataset_sizes['valid'], transforms=data_transforms['valid'])

    dataloaders ={}
    dataloaders['train'] = DataLoader(train_dataset,  batch_size=1, num_workers=6, shuffle=False)
    dataloaders['valid'] = DataLoader(train_dataset, batch_size=1, num_workers=6, shuffle=False)


    model_ft = models.resnet50(pretrained=True)
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs, 2)
    model_ft = model_ft.cuda()

    criterion = nn.CrossEntropyLoss()

    # all parameters are being optimized
    optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

    exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=4, gamma=0.1)

    model_ft = train_model(model_ft, dataloaders, dataset_sizes, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=epochs)
    
    torch.save(model_ft.state_dict(), save_path+str(i))

In [ ]:
save_path = '/media/rene/Data/camelyon_out/inline_batch_random/resnet50_10e_quarter_'

for i in range(5):
    normal_train_subset = random.sample(normal_train, int(len(normal_train)/4))

    train_dataset = WSIDataset(data_loc, normal_train_subset, tumor_train, batch_size, length=dataset_sizes['train'], transforms=data_transforms['train'])
    valid_dataset = WSIDataset(data_loc, normal_valid, tumor_valid, batch_size, length=dataset_sizes['valid'], transforms=data_transforms['valid'])

    dataloaders ={}
    dataloaders['train'] = DataLoader(train_dataset,  batch_size=1, num_workers=6, shuffle=False)
    dataloaders['valid'] = DataLoader(train_dataset, batch_size=1, num_workers=6, shuffle=False)

    model_ft = models.resnet50(pretrained=True)
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs, 2)
    model_ft = model_ft.cuda()

    criterion = nn.CrossEntropyLoss()

    # all parameters are being optimized
    optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

    exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=4, gamma=0.1)

    model_ft = train_model(model_ft, dataloaders, dataset_sizes, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=epochs)
    
    torch.save(model_ft.state_dict(), save_path+str(i))

# Test

In [8]:
import os
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data
import torchvision.models as models
from torchvision import datasets, models, transforms
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

data_loc = '/media/rene/Data/camelyon_out/tiles_224_100t'
batch_size = 32

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

image_datasets = {x: datasets.ImageFolder(os.path.join(data_loc, x),
                                          data_transforms[x])
                  for x in ['train', 'valid']}

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'valid']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'valid']}

###############

# model_ft= models.resnet50(pretrained=True)
# num_ftrs = model_ft.fc.in_features
# model_ft.fc = nn.Linear(num_ftrs, 2)
# model_ft = model_ft.cuda()

# model_ft.load_state_dict(torch.load('/media/rene/Data/camelyon_out/inline_batch_random/resnet50_10e_half_1.h5'))
# model_ft.eval()

# use_gpu = torch.cuda.is_available()

# # test_accu(model=model_ft, test_loader=dataloaders['valid'])

In [9]:
model_ft= models.resnet50(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 2)
model_ft = model_ft.cuda()

model_ft.load_state_dict(torch.load('/media/rene/Data/camelyon_out/inline_batch_random/resnet50_10e_full_0.h5'))
model_ft.eval()

use_gpu = torch.cuda.is_available()

test_accu(model=model_ft, test_loader=dataloaders['valid'])

Accu: 0.804251497005988


In [10]:
model_ft= models.resnet50(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 2)
model_ft = model_ft.cuda()

model_ft.load_state_dict(torch.load('/media/rene/Data/camelyon_out/inline_batch_random/resnet50_10e_full_1.h5'))
model_ft.eval()

use_gpu = torch.cuda.is_available()

test_accu(model=model_ft, test_loader=dataloaders['valid'])

Accu: 0.751317365269461


In [12]:
model_ft= models.resnet50(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 2)
model_ft = model_ft.cuda()

model_ft.load_state_dict(torch.load('/media/rene/Data/camelyon_out/inline_batch_random/resnet50_10e_full_2.h5'))
model_ft.eval()

use_gpu = torch.cuda.is_available()

test_accu(model=model_ft, test_loader=dataloaders['valid'])

Accu: 0.7355688622754492


## Debug

In [ ]:
model_ft.eval()
x,y=next(iter(dataloaders['valid']))
y = torch.cat(y)
x=torch.squeeze(x)
x = Variable(x).cuda()
y = Variable(y).cuda()

preds = model_ft(x)

del x, y, preds